# All imports

In [21]:
import pandas as pd
import string
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
from pickle import dump, load
from sklearn.feature_selection import chi2 , SelectKBest, mutual_info_regression
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, LassoCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, make_scorer, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn import tree

# Loading processed data

In [22]:
movies_final=pd.read_csv('/workspaces/KNN_movies_classf/data/processed/movies_final.csv')

In [29]:
movies_final.head()

,movie_id,title,tags
0,19995,Avatar,['In the 22nd century a paraplegic Marine is ...
1,285,Pirates of the Caribbean: At World's End,['Captain Barbossa long believed to be dead ...
2,206647,Spectre,['A cryptic message from Bond’s past sends him...
3,49026,The Dark Knight Rises,"[""Following the death of District Attorney Har..."
4,49529,John Carter,"[""John Carter is a war-weary former military ..."


# Vectorising data

In [24]:
vector=TfidfVectorizer()

tfid_matrix=vector.fit_transform(movies_final['tags'])

# Estimating KNN model

In [32]:
knn_base=NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine")
knn_base.fit(tfid_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

# Getting movie recommendations

In [27]:
def get_recom (movie_title):
    mov_index=movies_final[movies_final['title']==movie_title].index[0]
    distance, index= knn_base.kneighbors(tfid_matrix[mov_index])
    similar_movs=[(movies_final['title'][i],distance[0][j]) for j,i in enumerate(index[0])]
    return similar_movs[1:]

In [28]:
input_movie='Avatar'
recommend=get_recom(input_movie)

print("Movie recommendations '{}'".format(input_movie))
for movie,distance in recommend:
    print("- Film '{}'".format(movie))

Movie recommendations 'Avatar'
- Film 'Avatar'
- Film 'Avatar'
- Film 'Avatar'
- Film 'Avatar'
- Film 'Avatar'


Storing model:

In [ ]:
dump(knn_base, open('/workspaces/KNN_movies_classf/models/knn_n_neigh6_brute_cosine.sav', 'wb'))